In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
RANDOM_SEED = 42
DATA_PATH = "data/"
DATA_FILE = "processed_traffic.parquet"

In [3]:
data = pd.read_parquet(DATA_PATH + DATA_FILE)
X, y = data.drop(columns=["Attack Name", "Label"]), data["Label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((49742, 47), (12436, 47), (49742,), (12436,))

In [4]:
X_train.head()

,Src Port,Dst Port,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Mean,...,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Idle Mean,Idle Std
50459,1883,52303,588990,2,4,0.0,15.0,0.000000,0.000000,3.750000,...,0,0,509,502,0,32,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
43594,56478,18665,103667254,10,0,636.0,0.0,63.600000,54.738165,0.000000,...,0,0,14726,0,5,32,1.618428e+06,1.759747e+06,1.943871e+07,9.732065e+06
44213,45125,1883,110000796,16,16,178.0,4.0,11.125000,6.075909,0.250000,...,0,0,502,64,13,32,1.896835e+05,4.219112e+05,9.810381e+06,4.003378e+05
17134,43231,1883,119998680,17,17,194.0,4.0,11.411765,5.990188,0.235294,...,0,0,502,64,14,32,1.735722e+05,4.052646e+05,9.826317e+06,3.859519e+05
13320,40571,1883,61047003,5,4,17.0,4.0,3.400000,5.458938,1.000000,...,0,0,502,64,2,32,2.600000e+03,0.000000e+00,5.999980e+07,0.000000e+00


In [5]:
y_train.head()

50459    1
43594    1
44213    1
17134    0
13320    0
Name: Label, dtype: int64

In [6]:
# Normalize the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
X_train.head()

,Src Port,Dst Port,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Mean,...,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Idle Mean,Idle Std
0,-1.759531,1.793392,-1.315682,-0.133439,-0.134831,-0.063223,-0.029439,-0.331845,-0.339586,-0.169665,...,-0.084939,-0.014693,-0.435118,-0.222006,-0.124596,0.640622,-0.238747,-0.284902,-0.709551,-0.206578
1,1.306773,0.201716,0.701594,-0.119532,-0.143527,-0.061783,-0.029500,0.144995,0.123035,-0.205212,...,-0.084939,-0.014693,0.247840,-0.259501,-0.115646,0.640622,0.086920,0.386412,0.153447,4.085911
2,0.669137,-0.592371,0.825543,-0.109101,-0.108743,-0.062820,-0.029483,-0.248436,-0.288235,-0.202842,...,-0.084939,-0.014693,-0.435455,-0.254721,-0.101326,0.640622,-0.200578,-0.123950,-0.274011,-0.030003
3,0.562761,-0.592371,1.021205,-0.107363,-0.106569,-0.062783,-0.029483,-0.246285,-0.288960,-0.202981,...,-0.084939,-0.014693,-0.435455,-0.254721,-0.099536,0.640622,-0.203820,-0.130300,-0.273303,-0.036348
4,0.413364,-0.592371,-0.132499,-0.128224,-0.134831,-0.063184,-0.029483,-0.306354,-0.293450,-0.195733,...,-0.084939,-0.014693,-0.435455,-0.254721,-0.121016,0.640622,-0.238224,-0.284902,1.954191,-0.206578


In [7]:
import torch

# Convert to tensors
X_train_tensor, y_train_tensor = torch.tensor(X_train.values, dtype=torch.float32), torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor, y_test_tensor = torch.tensor(X_test.values, dtype=torch.float32), torch.tensor(y_test.values, dtype=torch.long)
X_train_tensor.shape, y_train_tensor.shape, X_test_tensor.shape, y_test_tensor.shape

(torch.Size([49742, 47]),
 torch.Size([49742]),
 torch.Size([12436, 47]),
 torch.Size([12436]))

In [8]:
import torch.nn as nn
import torch.nn.functional as F

class IDSModel(nn.Module):
    def __init__(self, input_size, layers,  hidden_size, num_classes):
        super(IDSModel, self).__init__()
        
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            *[layer for _ in range(layers) for layer in (nn.Linear(hidden_size, hidden_size), nn.ReLU())],
            nn.Linear(hidden_size, num_classes)
        )
    
    def forward(self, x):
        return self.net(x)

In [9]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score, roc_auc_score

def training_loop(dataloader, epochs, model, criterion, optimizer, quiet=True):
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        if not quiet:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader)}")
    if not quiet:
        print("Training complete.")

def train_model(X_train, y_train, input_size, layers, hidden_size, num_classes, batch_size=64, epochs=10, learning_rate=0.001, quiet=True):
    torch.manual_seed(RANDOM_SEED)

    dataset = TensorDataset(X_train, y_train)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    model = IDSModel(input_size, layers, hidden_size, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    training_loop(dataloader, epochs, model, criterion, optimizer, quiet=quiet)
    return model

def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, predicted = torch.max(outputs, 1)
    print(classification_report(y_test, predicted))
    return accuracy_score(y_test, predicted)

In [10]:
def init_training(X_train, y_train, input_size, layers, hidden_size, num_classes, batch_size=64, learning_rate=0.001, quiet=True):
    """
    Initialize needed variables to start training the model and return them.
    """
    torch.manual_seed(RANDOM_SEED)
    dataset = TensorDataset(X_train, y_train)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model = IDSModel(input_size, layers, hidden_size, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    return dataloader, model, criterion, optimizer

def train_for_epochs(dataloader, epochs, model, criterion, optimizer, quiet=True):
    """
    Train the model for a specified number of epochs.
    """
    training_loop(dataloader, epochs, model, criterion, optimizer, quiet=quiet)
    return dataloader, model, criterion, optimizer

def evaluate(model, X_test, y_test, quiet=True):
    """
    Evaluate the model on the test dataset and return accuracy, F1 score, recall, and AUC.
    """
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, predicted = torch.max(outputs, 1)
        prob_class1 = outputs[:, 1]  # Assuming binary classification for AUC calculation

    if not quiet:
        print(classification_report(y_test, predicted))

    return accuracy_score(y_test, predicted), f1_score(y_test, predicted), recall_score(y_test, predicted), roc_auc_score(y_test, prob_class1)


In [151]:
# Hyperparameter testing with different epochs, layers, and hidden sizes
MAX_EPOCHS = 50
layers_to_test = [1, 2, 3]
hidden_sizes_to_test = [32, 64, 128]

best_accuracy = 0
best_f1 = 0
best_recall = 0
best_auc = 0

best_models = {
    "accuracy": (0, 0, 0),
    "f1": (0, 0, 0),
    "recall": (0, 0, 0),
    "auc": (0, 0, 0)
}

for layers in layers_to_test:
    for hidden_size in hidden_sizes_to_test:
        dataloader, model, criterion, optimizer = init_training(
            X_train_tensor, y_train_tensor,
            input_size=X_train_tensor.shape[1],
            layers=layers,
            hidden_size=hidden_size,
            num_classes=len(y.unique()),
            batch_size=64,
            learning_rate=0.001,
            quiet=True
        )

        # Train in increments of 10 epochs and evaluate
        for i in range(MAX_EPOCHS // 10):
            dataloader, model, criterion, optimizer = train_for_epochs(
                dataloader, 10, model, criterion, optimizer, quiet=True
            )
            accuracy, f1, recall, auc = evaluate(model, X_test_tensor, y_test_tensor)
            print(f"Layers: {layers}, Hidden Size: {hidden_size}, Epochs: {(i+1)*10}, Accuracy: {accuracy}, F1: {f1}, Recall: {recall}, AUC: {auc}")

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_models["accuracy"] = (layers, hidden_size, (i+1)*10)
            if f1 > best_f1:
                best_f1 = f1
                best_models["f1"] = (layers, hidden_size, (i+1)*10)
            if recall > best_recall:
                best_recall = recall
                best_models["recall"] = (layers, hidden_size, (i+1)*10)
            if auc > best_auc:
                best_auc = auc
                best_models["auc"] = (layers, hidden_size, (i+1)*10)

for metric, (layers, hidden_size, epochs) in best_models.items():
    print(f"Best {metric.capitalize()} Model - Layers: {layers}, Hidden Size: {hidden_size}, Epochs: {epochs}")
            

Layers: 1, Hidden Size: 32, Epochs: 10, Accuracy: 0.8896751366999035, F1: 0.8745427944403804, Recall: 0.8023489932885906, AUC: 0.9485980890722999
Layers: 1, Hidden Size: 32, Epochs: 20, Accuracy: 0.9061595368285622, F1: 0.894798521590192, Recall: 0.8327181208053691, AUC: 0.9621899237660168
Layers: 1, Hidden Size: 32, Epochs: 30, Accuracy: 0.917175940816983, F1: 0.9083466809040754, Recall: 0.8563758389261745, AUC: 0.9652211987679858
Layers: 1, Hidden Size: 32, Epochs: 40, Accuracy: 0.9230459954969443, F1: 0.9165576772168454, Recall: 0.8818791946308725, AUC: 0.9706894014450879
Layers: 1, Hidden Size: 32, Epochs: 50, Accuracy: 0.9274686394339016, F1: 0.9200354609929078, Recall: 0.8706375838926175, AUC: 0.9745960562697166
Layers: 1, Hidden Size: 64, Epochs: 10, Accuracy: 0.8950627211321969, F1: 0.8821883181366796, Recall: 0.8197986577181208, AUC: 0.9539745099095887
Layers: 1, Hidden Size: 64, Epochs: 20, Accuracy: 0.9237697008684465, F1: 0.9155682223013893, Recall: 0.8624161073825504, AUC:

In [11]:
# Train the final model with the best hyperparameters found (2 layers, 64 hidden size) at a higher epoch count
FINAL_MAX_EPOCHS = 100
final_layers = 2
final_hidden_size = 64

final_dataloader, final_model, final_criterion, final_optimizer = init_training(
    X_train_tensor, y_train_tensor,
    input_size=X_train_tensor.shape[1],
    layers=final_layers,
    hidden_size=final_hidden_size,
    num_classes=len(y.unique()),
    batch_size=64,
    learning_rate=0.001,
    quiet=True
)

for i in range(FINAL_MAX_EPOCHS // 10):
    final_dataloader, final_model, final_criterion, final_optimizer = train_for_epochs(
        final_dataloader, 10, final_model, final_criterion, final_optimizer, quiet=True
    )
    accuracy, f1, recall, auc = evaluate(final_model, X_test_tensor, y_test_tensor)
    print(f"Final Model (2 layers, 64 hidden size) - Epochs: {(i+1)*10}, Accuracy: {accuracy}, F1: {f1}, Recall: {recall}, AUC: {auc}")

Final Model (2 layers, 64 hidden size) - Epochs: 10, Accuracy: 0.9121100032164683, F1: 0.9023845672948111, Recall: 0.8476510067114094, AUC: 0.9698927324846309
Final Model (2 layers, 64 hidden size) - Epochs: 20, Accuracy: 0.9409778063686073, F1: 0.935973482205164, Recall: 0.9001677852348994, AUC: 0.978924298701245
Final Model (2 layers, 64 hidden size) - Epochs: 30, Accuracy: 0.9388871019620457, F1: 0.9329451208752426, Recall: 0.8870805369127517, AUC: 0.9819518687482124
Final Model (2 layers, 64 hidden size) - Epochs: 40, Accuracy: 0.9484560952074622, F1: 0.9444877457348229, Recall: 0.9149328859060403, AUC: 0.986384575365521
Final Model (2 layers, 64 hidden size) - Epochs: 50, Accuracy: 0.9570601479575426, F1: 0.954092159559835, Recall: 0.9310402684563759, AUC: 0.9900077881781362
Final Model (2 layers, 64 hidden size) - Epochs: 60, Accuracy: 0.9536024445159215, F1: 0.9498740335331423, Recall: 0.9172818791946309, AUC: 0.9890192258664932
Final Model (2 layers, 64 hidden size) - Epochs: 7

### Result
From the initial sieve and further refinement on the final model, we find that a 2 layer, 64 hidden unit model trained up to 50 epochs performs the best according to our criteria.